You can tune 2+ models using the same grid search! Here's how:

Create multiple parameter dictionaries
Specify the model within each dictionary
Put the dictionaries in a list

In [3]:
import pandas as pd
df = pd.read_csv('train.csv')

In [4]:
cols = ['Sex', 'Name', 'Age']
X = df[cols]
y = df['Survived']

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [6]:
# this will be the first Pipeline step
ct = ColumnTransformer(
    [('ohe', OneHotEncoder(), ['Sex']),
     ('vectorizer', CountVectorizer(), 'Name'),
     ('imputer', SimpleImputer(), ['Age'])])

In [7]:
# each of these models will take a turn as the second Pipeline step
clf1 = LogisticRegression(solver='liblinear', random_state=1)
clf2 = RandomForestClassifier(random_state=1)

In [8]:
# create the Pipeline
pipe = Pipeline([('preprocessor', ct), ('classifier', clf1)])

In [9]:
# create the parameter dictionary for clf1
params1 = {}
params1['preprocessor__vectorizer__ngram_range'] = [(1, 1), (1, 2)]
params1['classifier__penalty'] = ['l1', 'l2']
params1['classifier__C'] = [0.1, 1, 10]
params1['classifier'] = [clf1]

In [10]:
# create the parameter dictionary for clf2
params2 = {}
params2['preprocessor__vectorizer__ngram_range'] = [(1, 1), (1, 2)]
params2['classifier__n_estimators'] = [100, 200]
params2['classifier__min_samples_leaf'] = [1, 2]
params2['classifier'] = [clf2]

In [11]:
# create a list of parameter dictionaries
params = [params1, params2]

In [12]:
# this will search every parameter combination within each dictionary
grid = GridSearchCV(pipe, params)
grid.fit(X, y)
grid.best_params_

{'classifier': LogisticRegression(random_state=1, solver='liblinear'),
 'classifier__C': 10,
 'classifier__penalty': 'l1',
 'preprocessor__vectorizer__ngram_range': (1, 2)}